In [15]:
import boto3
import pandas as pd; pd.set_option('display.max_column', 100)
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import recall_score

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

## define bucket in which you are trying to reach
s3 = boto3.resource('s3')
bucket_name = 'daltondencklau-data445-bucket'
bucket = s3.Bucket(bucket_name)

## define csv file to read in the bucket
file_key= 'framingham.csv'

## syntax to allow us to read the file
bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

## reading the data file
heart = pd.read_csv(file_content_stream)
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [16]:
## dropping missing values
heart = heart.dropna()

In [17]:
## defining input and target variables
X = heart[['age', 'currentSmoker', 'totChol', 'BMI', 'heartRate']]
Y = heart['TenYearCHD']

In [18]:
## when you work with tree models, you do not need to standardize the data (even though assignment says to)

In [19]:
## splitting the data into 80% training and 20% testing (y = stratified)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y)

In [22]:
## building models: adaboost and random forest
ada_md = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth = 3), n_estimators = 500, learning_rate = 0.1).fit(X_train, Y_train)
rf_md = RandomForestClassifier(n_estimators = 500, max_depth = 3).fit(X_train, Y_train)

## predicting on the testing data test, extracting values = 1
ada_preds = ada_md.predict_proba(X_test)[:, 1]
rf_preds = rf_md.predict_proba(X_test)[:, 1]

## changing likelihoods into labels
ada_labels = np.where(ada_preds < 0.1, 0, 1)
rf_labels = np.where(rf_preds < 0.1, 0, 1)

## printing results
print('The recall score for the AdaBoost model is', recall_score(Y_test, ada_labels))
print('The recall score for the Random Forest model is', recall_score(Y_test, rf_labels))

The recall score for the AdaBoost model is 1.0
The recall score for the Random Forest model is 0.9017857142857143


I would use the AdaBoost model to predict the risk of coronary disease because it has the highest recall score.